In [1]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [2]:
import networkx as nx
import matplotlib.pyplot as plt

query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(n:IngredientResearch)
RETURN p.name AS Product, r.ingredients_orig AS Recipe, n.ingredient_name AS IngredientResearch
"""
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)


fig = px.scatter(df, x="IngredientResearch", color="Recipe", title="Recipe and IngredientResearch")
fig.show()


In [3]:
query = """
MATCH (r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(n:IngredientResearch)
RETURN r.ingredients_orig AS Recipe, n.ingredient_name AS IngredientResearch
"""
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)



ingredientresearch_recipe_counts = df.groupby('IngredientResearch')['Recipe'].nunique().reset_index()
ingredientresearch_recipe_counts.columns = ['IngredientResearch', 'RecipeCount']

import plotly.express as px

fig = px.bar(ingredientresearch_recipe_counts, x='IngredientResearch', y='RecipeCount',
             color='RecipeCount', color_continuous_scale='Viridis',
             hover_name='IngredientResearch', 
             labels={'RecipeCount': 'Number of Recipes'})

fig.update_layout(title='Number of Recipes per Ingredient Research',
                   xaxis_title='Ingredient Research',
                   yaxis_title='Number of Recipes')

fig.show()


In [4]:
import plotly.express as px

query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, n.nutrient AS NutritionalInformation
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

# calculate each IngredientResearch conclude the NutritionalInformation number
nutriinfo_counts = df.groupby('IngredientResearch')['NutritionalInformation'].nunique().reset_index()
nutriinfo_counts.columns = ['IngredientResearch', 'NutriInfoCount']


fig = px.bar(nutriinfo_counts, x='IngredientResearch', y='NutriInfoCount',
             color='NutriInfoCount', color_continuous_scale='Viridis',
             hover_name='IngredientResearch',
             labels={'NutriInfoCount': 'Number of Nutritional Information'})


fig.update_layout(
    title='Number of Nutritional Information per Ingredient Research',
    xaxis_title='Ingredient Research',
    yaxis_title='Number of Nutritional Information',
    font=dict(family='Arial', size=12, color='rgb(60,60,100)'))


fig.show()

In [5]:
query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, n.nutrient AS NutritionalInformation
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

ingredientresearch_nutriinfo = df.groupby('IngredientResearch')['NutritionalInformation'].apply(list).reset_index()

from itertools import combinations
from collections import Counter

nutriinfo_pairs = []
for nutriinfos in ingredientresearch_nutriinfo['NutritionalInformation']:
    pairs = combinations(nutriinfos, 5)
    nutriinfo_pairs.extend(pairs)

nutriinfo_pair_counts = Counter(nutriinfo_pairs)
nutriinfo_pair_df = pd.DataFrame(nutriinfo_pair_counts.items(), columns=['NutriInfoPair', 'Count'])

import plotly.express as px

fig = px.bar(nutriinfo_pair_df.sort_values('Count', ascending=False).head(40),
             x='NutriInfoPair', y='Count', color='Count',
             color_continuous_scale='Viridis', title='Nutritional Information Co-occurrence in Ingredient Research',
             labels={'NutriInfoPair': 'Nutritional Information Pair', 'Count': 'Number of Co-occurrences'})

fig.show()


In [6]:
import plotly.express as px

query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, n.nutrient AS NutritionalInformation
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)


ingredientresearch_nutriinfo = df.groupby('IngredientResearch')['NutritionalInformation'].apply(list).reset_index()

import plotly.graph_objects as go


ingredientresearch_nutriinfo = df.groupby('IngredientResearch')['NutritionalInformation'].apply(list).reset_index()

import plotly.express as px


counts = df.groupby(['IngredientResearch', 'NutritionalInformation']).size().reset_index(name='Count')


fig = px.density_heatmap(counts, x='IngredientResearch', y='NutritionalInformation', z='Count', 
                         title='Relationship between Ingredient Research and Nutritional Information')

fig.show()


fig = px.bar(counts, x='IngredientResearch', y='Count', color='NutritionalInformation', 
             title='Relationship between Ingredient Research and Nutritional Information')

fig.show()







In [7]:
import plotly.express as px

query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, n.nutrient AS NutritionalInformation
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)


ingredientresearch_nutriinfo = df.groupby('NutritionalInformation')['IngredientResearch'].apply(list).reset_index()

import plotly.graph_objects as go



import plotly.express as px




fig = px.bar(counts, x='NutritionalInformation', y='Count', color='IngredientResearch', 
             title='Relationship between Nutritional Information and Ingredient Research')

fig.show()

In [8]:
query = """
MATCH (r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(n:IngredientResearch)
RETURN r.ingredients_orig AS Recipe, n.ingredient_name AS IngredientResearch
"""
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)


recipe_ingredients = df.groupby('Recipe')['IngredientResearch'].apply(list).reset_index()

from itertools import combinations
from collections import Counter

ingredient_pairs = []
for ingredients in recipe_ingredients['IngredientResearch']:
    pairs = combinations(ingredients, 6)
    ingredient_pairs.extend(pairs)

ingredient_pair_counts = Counter(ingredient_pairs)
ingredient_pair_df = pd.DataFrame(ingredient_pair_counts.items(), columns=['IngredientPair', 'Count'])

import plotly.express as px

fig = px.bar(ingredient_pair_df.sort_values('Count', ascending=False).head(20),
             x='IngredientPair', y='Count', color='Count',
             color_continuous_scale='Viridis', title='Ingredient Co-occurrence in Recipes',
             labels={'IngredientPair': 'Ingredient Pair', 'Count': 'Number of Co-occurrences'})

fig.show()

In [9]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (d:DietarySuitability)
RETURN d.diet AS Diet, d.suitability AS Suitability
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

# Caculate each dietarysuitability  diet count
diet_counts = df.groupby("Suitability")["Diet"].value_counts().reset_index(name="Count")

# use Plotly to create bar chart
fig = px.bar(diet_counts, x="Diet", y="Count", color="Suitability", title="Diet Distribution for Each DietarySuitability")
fig.show()


In [10]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (d:DietarySuitability)
RETURN d.diet AS Diet, d.suitability AS Suitability
"""

import plotly.express as px


plot_data = diet_counts.pivot(index='Diet', columns='Suitability', values='Count').reset_index().rename_axis(None, axis=1)


fig = px.bar(plot_data, x='Diet', y=plot_data.columns[1:], title='Diet Distribution for Each Dietary Suitability')


fig.update_layout(
    xaxis_title='Diet Type',
    yaxis_title='Count',
    legend_title='Dietary Suitability',
    barmode='group'
)


fig.show()

In [35]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (r:IngredientResearch)-[:HAS_DIETARY_SUITABILITY]->(i:DietarySuitability)
RETURN r.ingredient_name AS IngredientResearch, i.diet AS DietarySuitability
"""



with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

import plotly.express as px

fig = px.scatter(df, y='IngredientResearch', x='DietarySuitability',
                 color='IngredientResearch',
                 hover_name='DietarySuitability', 
                 hover_data=['IngredientResearch'])

fig.update_layout(title='Ingredient Research and Dietary Suitability', 
                  yaxis_title='Ingredient Research',
                  xaxis_title='Dietary Suitability',
                  legend_title='Dietary Suitability')

fig.show()



In [36]:
query = """
MATCH (r:IngredientResearch)-[:HAS_DIETARY_SUITABILITY]->(i:DietarySuitability)
RETURN r.ingredient_name AS IngredientResearch, i.diet AS DietarySuitability
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

import plotly.express as px

fig = px.scatter(df, x='IngredientResearch', y='DietarySuitability',
                 color='DietarySuitability',
                 hover_name='IngredientResearch', 
                 hover_data=['DietarySuitability'])

fig.update_layout(title='Ingredient Research and Dietary Suitability', 
                  xaxis_title='Ingredient Research',
                  yaxis_title='Dietary Suitability',
                  legend_title='Dietary Suitability')

fig.show()

In [37]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (r:IngredientResearch)-[:HAS_DIETARY_SUITABILITY]->(i:DietarySuitability)
RETURN r.ingredient_name AS IngredientResearch, i.suitability AS DietarySuitability
"""



with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

import plotly.express as px

fig = px.scatter(df, y='IngredientResearch', x='DietarySuitability',
                 color='DietarySuitability',
                 hover_name='DietarySuitability', 
                 hover_data=['IngredientResearch'])

fig.update_layout(title='Ingredient Research and Dietary Suitability', 
                  yaxis_title='Ingredient Research',
                  xaxis_title='Dietary Suitability',
                  legend_title='DietarySuitability')

fig.show()

In [38]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (r:IngredientResearch)-[:HAS_DIETARY_SUITABILITY]->(i:DietarySuitability)
RETURN r.ingredient_name AS IngredientResearch, i.suitability AS DietarySuitability
"""



with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

import plotly.express as px

fig = px.scatter(df, x='IngredientResearch', y='DietarySuitability',
                 color='DietarySuitability',
                 hover_name='IngredientResearch', 
                 hover_data=['DietarySuitability'])

fig.update_layout(title='Ingredient Research and Dietary Suitability', 
                  xaxis_title='Ingredient Research',
                  yaxis_title='Dietary Suitability',
                  legend_title='Dietary Suitability')

fig.show()

In [34]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (r:IngredientResearch)-[:HAS_DIETARY_SUITABILITY]->(i:DietarySuitability)
RETURN r.ingredient_name AS IngredientResearch, i.suitability AS DietarySuitability, i.diet AS DietaryDiet
"""



with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

import plotly.express as px


df['DietarySuitability'] = df['DietaryDiet'] + ' / ' + df['DietarySuitability']

import plotly.express as px


fig = px.scatter(df, y='IngredientResearch', x='DietarySuitability',
                 color='DietarySuitability',
                 hover_name='IngredientResearch', 
                 hover_data=['DietaryDiet', 'DietarySuitability'])


fig.update_layout(
    title='Ingredient Research and Dietary Suitability',
    yaxis_title='Ingredient Research',
    xaxis_title='Dietary Suitability',
    legend_title='Dietary Suitability'
)


fig.show()

In [16]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px

query = """
MATCH (r:IngredientResearch)-[:HAS_DIETARY_SUITABILITY]->(i:DietarySuitability)
RETURN r.ingredient_name AS IngredientResearch, i.suitability AS DietarySuitability, i.diet AS DietaryDiet
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

df['DietarySuitability'] = df['DietaryDiet'] + ' / ' + df['DietarySuitability']


fig = px.bar(df, x='IngredientResearch', y='DietarySuitability',
             color='DietarySuitability',
             hover_data=['DietaryDiet', 'DietarySuitability'],
             title='Ingredient Research and Dietary Suitability',
             labels={'DietarySuitability': 'Dietary Suitability'})


fig.update_layout(
    xaxis_title='Ingredient Research',
    yaxis_title='Dietary Suitability',
    legend_title='Dietary Suitability'
)


fig.show()


In [39]:
query = """
MATCH (p:Product)-[:HAS_RECIPE]->(i:Recipe)-[:USES_INGREDIENT]->(N:Ingredient)-[:HAS_RESEARCH]->(a:IngredientResearch)
RETURN p.name AS Product, a.ingredient_name AS IngredientResearch
"""


with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

fig = px.scatter(df, x='Product', y='IngredientResearch', color='IngredientResearch',
                 title='Ingredient in Products',
                 labels={'IngredientResearch': 'Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title='Ingredients'
)

fig.show()



In [40]:
query = """
MATCH (p:Product)-[:HAS_RECIPE]->(i:Recipe)-[:USES_INGREDIENT]->(N:Ingredient)-[:HAS_RESEARCH]->(a:IngredientResearch)
RETURN p.name AS Product, a.ingredient_name AS IngredientResearch
"""


with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

fig = px.scatter(df, y='Product', x='IngredientResearch', color='IngredientResearch',
                 title='Ingredient in Products',
                 labels={'IngredientResearch': 'Ingredients'})

fig.update_layout(
    yaxis_title='Product',
    xaxis_title='Ingredients'
)

fig.show()


In [41]:
fig = px.scatter(df, x='Product', y='IngredientResearch', color='IngredientResearch',
                 title='Ingredient in Products',
                 labels={'IngredientResearch': 'Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title=' Ingredients'
)

fig.show()


In [20]:
fig = px.bar(df, x='Product', y='IngredientResearch', color='Product',
             title='Ingredient in Products',
             labels={'IngredientResearch': ' Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title='Ingredients'
)

fig.show()


In [21]:
fig = px.bar(df, y='Product', x='IngredientResearch', color='IngredientResearch',
             title='Ingredient in Products',
             labels={'IngredientResearch': ' Ingredients'})

fig.update_layout(
    yaxis_title='Product',
    xaxis_title=' Ingredients'
)

fig.show()


In [22]:
fig = px.bar(df, x='Product', y='IngredientResearch', color='IngredientResearch',
             title='Ingredients in Products',
             labels={'IngredientResearch': 'Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title=' Ingredients'
)

fig.show()


In [30]:
fig = px.bar(df, x='Product', y='IngredientResearch', color='IngredientResearch',
             title='Ingredients in Products',
             labels={'IngredientResearch': ' Ingredients'})

fig.update_layout(
    xaxis_title='Product',
    yaxis_title='Ingredients',
    xaxis_showticklabels=False,  # hide x label
    yaxis_showticklabels=False  # hide x label
)

fig.show()


In [42]:
fig = px.scatter(df, y='Product', x='IngredientResearch', color='IngredientResearch',
             title='Ingredients in Products',
             labels={'IngredientResearch': ' Ingredients'})

fig.update_layout(
    yaxis_title='Product',
    xaxis_title='Ingredients',
    xaxis_showticklabels=False  # hide x label
)

fig.show()